In [1]:
import datetime

import pandas as pd
pd.set_option('display.max_columns', None)

In [4]:
list_strings = []
with open(r"data/DadosBO_2022_1(FURTO DE CELULAR).xls", encoding="iso-8859-1") as openfile:
    for line in openfile:
        list_strings.append(line)

In [71]:
list_df = [x.replace("\x00", "").replace("\n","").split("\t") for x in list_strings]

df_00 = pd.DataFrame(list_df)

new_header = df_00.iloc[0]
df_00 = df_00[1:]
df_00.columns = new_header
df_00 = df_00.reset_index(drop=True)

mask = pd.Series([x is not None for x in df_00["NUMERO_BOLETIM"]])
df_00 = df_00[mask]


In [72]:
df_00

,ANO_BO,NUM_BO,NUMERO_BOLETIM,BO_INICIADO,BO_EMITIDO,DATAOCORRENCIA,HORAOCORRENCIA,PERIDOOCORRENCIA,DATACOMUNICACAO,DATAELABORACAO,BO_AUTORIA,FLAGRANTE,NUMERO_BOLETIM_PRINCIPAL,LOGRADOURO,NUMERO,BAIRRO,CIDADE,UF,LATITUDE,LONGITUDE,DESCRICAOLOCAL,EXAME,SOLUCAO,DELEGACIA_NOME,DELEGACIA_CIRCUNSCRICAO,ESPECIE,RUBRICA,DESDOBRAMENTO,STATUS,TIPOPESSOA,VITIMAFATAL,NATURALIDADE,NACIONALIDADE,SEXO,DATANASCIMENTO,IDADE,ESTADOCIVIL,PROFISSAO,GRAUINSTRUCAO,CORCUTIS,NATUREZAVINCULADA,TIPOVINCULO,RELACIONAMENTO,PARENTESCO,PLACA_VEICULO,UF_VEICULO,CIDADE_VEICULO,DESCR_COR_VEICULO,DESCR_MARCA_VEICULO,ANO_FABRICACAO,ANO_MODELO,DESCR_TIPO_VEICULO,QUANT_CELULAR,MARCA_CELULAR
0,2021,2528434,2528434/2021,31/12/2021 23:20:07,01/01/2022 00:33:43,30/12/2021,17:30,A TARDE,31/12/2021,31/12/2021 23:20:07,Desconhecida,Não,,AVENIDA DO ESTADO,5018,CAMBUCI,S.PAULO,SP,"-23,5572300039987","-46,620146983325",Via pública,,ENCAMINHAMENTO DP ÁREA DO FATO,DELEGACIA ELETRONICA,08º D.P. BRAS,Título II - Patrimônio (arts. 155 a 183),Furto (art. 155) - INTERIOR DE VEICULO,,Consumado,,,,,,,,,,,,,,,,,,,,,0,0,,1,
1,2022,37,37/2022,01/01/2022 01:01:51,01/01/2022 01:15:15,31/12/2021,21:10,A NOITE,01/01/2022,01/01/2022 01:01:51,Desconhecida,Não,,RUA CANTAREIRA,306,CENTRO HISTÓRICO DE SÃO PAULO,S.PAULO,SP,"-23,541819","-46,629386",Via pública,,ENCAMINHAMENTO DP ÁREA DO FATO,DELEGACIA ELETRONICA,01º D.P. SE,Título II - Patrimônio (arts. 155 a 183),Furto (art. 155) - INTERIOR DE VEICULO,,Consumado,,,,,,,,,,,,,,,,,,,,,0,0,,1,
2,2022,1,1/2022,01/01/2022 01:23:59,01/01/2022 01:38:37,01/01/2022,00:05,DE MADRUGADA,01/01/2022,01/01/2022 01:23:59,Desconhecida,Não,,RUA antonio rodrigues,1,VILA COMERCIAL,PRESIDENTE PRUDENTE,SP,"-22,123472233","-51,399824479",Via pública,,BO PARA REGISTRO,DEL.SEC.PRES.PRUDENTE PLANTÃO,02º D.P. PRES. PRUDENTE,Título II - Patrimônio (arts. 155 a 183),Furto (art. 155) - TRANSEUNTE,,Consumado,,,,,,,,,,,,,,,,,,,,,0,0,,1,Motorola
3,2022,2,2/2022,01/01/2022 01:52:45,01/01/2022 02:06:38,31/12/2021,23:30,A NOITE,01/01/2022,01/01/2022 01:52:45,Conhecida,Sim,,AVENIDA PAULISTA,2444,JARDIM PAULISTA,S.PAULO,SP,"-23,55635981875","-46,6624223639285",Via pública,,ENCAMINHAMENTO DP ÁREA DO FATO,36º D.P. VILA MARIANA,78º D.P. JARDINS,Título II - Patrimônio (arts. 155 a 183),Furto (art. 155) - TRANSEUNTE,§ 4º IV - Mediante concurso de duas ou mais pe...,Consumado,,,,,,,,,,,,,,,,,,,,,0,0,,1,Samsung
4,2022,114,114/2022,01/01/2022 02:33:50,01/01/2022 02:33:49,01/01/2022,,EM HORA INCERTA,01/01/2022,01/01/2022 02:33:50,Desconhecida,Não,,RUA RIO DE JANEIRO,50,LOTEAMENTO SILEX,GUARUJA,SP,"-23,9967395084","-46,260999049",Via Pública,,BO PARA INVESTIGAÇÃO,DELEGACIA ELETRONICA,DEL.POL.GUARUJA,Título II - Patrimônio (arts. 155 a 183),Furto (art. 155) - OUTROS,,Consumado,,,,,,,,,,,,,,,,,,,,,0,0,,,SAMSUNG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13495,2022,233237,233237/2022,31/01/2022 23:45:07,31/01/2022 23:45:33,29/01/2022,,EM HORA INCERTA,31/01/2022,31/01/2022 23:45:07,Desconhecida,Não,,AVENIDA PRESTES MAIA,0,CENTRO,S.PAULO,SP,,,Via Pública,,BO PARA INVESTIGAÇÃO,DELEGACIA ELETRONICA,03º D.P. CAMPOS ELISEOS,Título II - Patrimônio (arts. 155 a 183),Furto (art. 155) - OUTROS,,Consumado,,,,,,,,,,,,,,,,,,,,,0,0,,,APPLE
13496,2022,233238,233238/2022,31/01/2022 23:45:51,31/01/2022 23:45:57,31/01/2022,,EM HORA INCERTA,31/01/2022,31/01/2022 23:45:51,Desconhecida,Não,,AVENIDA NORDESTINA,5700,LAJEADO,S.PAULO,SP,"-23,5278829356689","-46,4178017188377",Via Pública,,BO PARA INVESTIGAÇÃO,DELEGACIA ELETRONICA,68º D.P. LAGEADO,Título II - Patrimônio (arts. 155 a 183),Furto (art. 155) - OUTROS,,Consumado,,,,,,,,,,,,,,,,,,,,,0,0,,,SAMSUNG
13497,2022,233298,233298/2022,31/01/2022 23:52:20,31/01/2022 23:51:53,31/01/2022,,EM HORA INCERTA,31/01/2022,31/01/2022 23:52:20,Desconhecida,Não,,AVENIDA SAO JOAO,920,REPUBLICA,S.PAULO,SP,"-

In [66]:
def treat_datetime_variable(datetime_variable_orig):
    datetime_variable_00 = [x.split(" ") for x  in datetime_variable_orig]
    datetime_variable_datetime = [[int(z) for z in x[0].split("/") + x[1].split(":")] for x in datetime_variable_00]
    datetime_variable_datetime = [datetime.datetime(d[2], d[1], d[0], hour=d[3], minute=d[4], second=d[5]) if d is not None else None for d in datetime_variable_datetime]

    return datetime_variable_datetime

In [67]:
def treat_date_variable(date_variable_orig):
    date_variable_orig_00 = [x.split("/") for x in date_variable_orig]
    date_variable_orig_01 = [[int(z) for z in x] for x in date_variable_orig_00]
    date_variable_orig_02 = [datetime.date(d[2], d[1], d[0]) for d in date_variable_orig_01]
    
    return date_variable_orig_02

In [68]:
def treat_datetime_ocorrencia(dataocorrencia, horaocorrencia):
    return [
        datetime.datetime.strptime(x + " " + y, "%d/%m/%Y %H:%M") if y != "" else 
        datetime.datetime.strptime(x, "%d/%m/%Y") for x,y in 
        zip(dataocorrencia, horaocorrencia)]

In [69]:
def treat_location(logradouro, numero, bairro, cidade, uf):
    return [x[0]+", "+x[1]+" "+x[2]+" "+x[3]+" - "+x[4]  for x in zip(logradouro, numero, bairro, cidade, uf)]

In [93]:
df_01 = df_00.copy()
df_01["id_report"] = df_00["NUMERO_BOLETIM"].astype(str)
df_01["date_emission"] = treat_datetime_variable(df_00["BO_EMITIDO"])
df_01["datetime_incident"] = treat_datetime_ocorrencia(df_01["DATAOCORRENCIA"], df_01["HORAOCORRENCIA"])
df_01["location"] = treat_location(df_00["LOGRADOURO"],df_00["NUMERO"],df_00["BAIRRO"],df_00["CIDADE"],df_00["UF"])
df_01["lat"] = [float(x.replace(",",".")) if x != "" else None for x in df_00["LATITUDE"]]
df_01["lon"] = [float(x.replace(",",".")) if x != "" else None for x in df_00["LONGITUDE"]]

df_01 = df_01.drop(
    ["ANO_BO", "NUM_BO", "NUMERO_BOLETIM", "BO_EMITIDO", "BO_INICIADO", "DATAOCORRENCIA","HORAOCORRENCIA", "PERIDOOCORRENCIA", 
    "DATACOMUNICACAO", "DATAELABORACAO", "BO_AUTORIA", "FLAGRANTE", 
    "NUMERO_BOLETIM_PRINCIPAL", "LOGRADOURO", "NUMERO", "BAIRRO", "CIDADE", "UF",
    "LATITUDE", "LONGITUDE", "DESCRICAOLOCAL" ,"EXAME", "SOLUCAO", "DELEGACIA_NOME", 
    "DELEGACIA_CIRCUNSCRICAO", "ESPECIE", "RUBRICA", "DESDOBRAMENTO", 
    "STATUS", "TIPOPESSOA", "VITIMAFATAL", "NATURALIDADE", "NACIONALIDADE", "SEXO",
    "DATANASCIMENTO", "IDADE", "ESTADOCIVIL", "PROFISSAO", "GRAUINSTRUCAO", "CORCUTIS",
    "NATUREZAVINCULADA", "TIPOVINCULO", "RELACIONAMENTO", "PARENTESCO", "PLACA_VEICULO",
    "UF_VEICULO", "CIDADE_VEICULO", "DESCR_COR_VEICULO", "DESCR_MARCA_VEICULO", 
    "ANO_FABRICACAO", "ANO_MODELO", "DESCR_TIPO_VEICULO", "QUANT_CELULAR", "MARCA_CELULAR"], 
    axis=1).copy()

# this is not an elegant solution, but using feathre or 
df_01["date_emission"] = df_01["date_emission"].astype(str)
df_01["datetime_incident"] = df_01["datetime_incident"].astype(str)

In [94]:
pd.DataFrame({x:[type(y)] for x,y in zip(df_01.iloc[0,:].index,list(df_01.iloc[0,:]))}).T

,0
id_report,<class 'str'>
date_emission,<class 'str'>
datetime_incident,<class 'str'>
location,<class 'str'>
lat,<class 'numpy.float64'>
lon,<class 'numpy.float64'>


In [95]:
df_01

,id_report,date_emission,datetime_incident,location,lat,lon
0,2528434/2021,2022-01-01 00:33:43,2021-12-30 17:30:00,"AVENIDA DO ESTADO, 5018 CAMBUCI S.PAULO - SP",-23.557230,-46.620147
1,37/2022,2022-01-01 01:15:15,2021-12-31 21:10:00,"RUA CANTAREIRA, 306 CENTRO HISTÓRICO DE SÃO PA...",-23.541819,-46.629386
2,1/2022,2022-01-01 01:38:37,2022-01-01 00:05:00,"RUA antonio rodrigues, 1 VILA COMERCIAL PRESID...",-22.123472,-51.399824
3,2/2022,2022-01-01 02:06:38,2021-12-31 23:30:00,"AVENIDA PAULISTA, 2444 JARDIM PAULISTA S.PAULO...",-23.556360,-46.662422
4,114/2022,2022-01-01 02:33:49,2022-01-01 00:00:00,"RUA RIO DE JANEIRO, 50 LOTEAMENTO SILEX GUARUJ...",-23.996740,-46.260999
...,...,...,...,...,...,...
13495,233237/2022,2022-01-31 23:45:33,2022-01-29 00:00:00,"AVENIDA PRESTES MAIA, 0 CENTRO S.PAULO - SP",NaN,NaN
13496,233238/2022,2022-01-31 23:45:57,2022-01-31 00:00:00,"AVENIDA NORDESTINA, 5700 LAJEADO S.PAULO - SP",-23.527883,-46.417802
13497,233298/2022,2022-01-31 23:51:53,2022-01-31 00:00:00,"AVENIDA SAO JOAO, 920 REPUBLICA S.PAULO - SP",-23.540591,-46.643470
13498,233241/2022,2022-01-31 23:55:46,2022-01-31 19:30:00,"AVENIDA VINTE E TRES DE MAIO, 1000 VILA MARIAN...",-23.576289,-46.643024


In [96]:
import pyarrow.feather as feather
# https://arrow.apache.org/docs/python/feather.html
# convert values to apache feather

# https://towardsdatascience.com/stop-using-csvs-for-storage-here-are-the-top-5-alternatives-e3a7c9018de0

In [97]:
feather.write_feather(df_01, "data/DadosBO_2022_1(FURTO DE CELULAR).feather")

In [98]:
# Result is pandas.DataFrame
df_02 = feather.read_feather("data/DadosBO_2022_1(FURTO DE CELULAR).feather")

# I'm having some problems using feather
# problems with date variables

In [99]:
df_02

,id_report,date_emission,datetime_incident,location,lat,lon
0,2528434/2021,2022-01-01 00:33:43,2021-12-30 17:30:00,"AVENIDA DO ESTADO, 5018 CAMBUCI S.PAULO - SP",-23.557230,-46.620147
1,37/2022,2022-01-01 01:15:15,2021-12-31 21:10:00,"RUA CANTAREIRA, 306 CENTRO HISTÓRICO DE SÃO PA...",-23.541819,-46.629386
2,1/2022,2022-01-01 01:38:37,2022-01-01 00:05:00,"RUA antonio rodrigues, 1 VILA COMERCIAL PRESID...",-22.123472,-51.399824
3,2/2022,2022-01-01 02:06:38,2021-12-31 23:30:00,"AVENIDA PAULISTA, 2444 JARDIM PAULISTA S.PAULO...",-23.556360,-46.662422
4,114/2022,2022-01-01 02:33:49,2022-01-01 00:00:00,"RUA RIO DE JANEIRO, 50 LOTEAMENTO SILEX GUARUJ...",-23.996740,-46.260999
...,...,...,...,...,...,...
13495,233237/2022,2022-01-31 23:45:33,2022-01-29 00:00:00,"AVENIDA PRESTES MAIA, 0 CENTRO S.PAULO - SP",NaN,NaN
13496,233238/2022,2022-01-31 23:45:57,2022-01-31 00:00:00,"AVENIDA NORDESTINA, 5700 LAJEADO S.PAULO - SP",-23.527883,-46.417802
13497,233298/2022,2022-01-31 23:51:53,2022-01-31 00:00:00,"AVENIDA SAO JOAO, 920 REPUBLICA S.PAULO - SP",-23.540591,-46.643470
13498,233241/2022,2022-01-31 23:55:46,2022-01-31 19:30:00,"AVENIDA VINTE E TRES DE MAIO, 1000 VILA MARIAN...",-23.576289,-46.643024


In [ ]:
# https://towardsdatascience.com/stop-using-csvs-for-storage-here-are-the-top-5-alternatives-e3a7c9018de0

In [100]:
df_01.to_parquet("data/DadosBO_2022_1(FURTO DE CELULAR).parquet")

In [101]:
df_02 = pd.read_parquet("data/DadosBO_2022_1(FURTO DE CELULAR).parquet")
# we got the same problem...

In [102]:
df_02

,id_report,date_emission,datetime_incident,location,lat,lon
0,2528434/2021,2022-01-01 00:33:43,2021-12-30 17:30:00,"AVENIDA DO ESTADO, 5018 CAMBUCI S.PAULO - SP",-23.557230,-46.620147
1,37/2022,2022-01-01 01:15:15,2021-12-31 21:10:00,"RUA CANTAREIRA, 306 CENTRO HISTÓRICO DE SÃO PA...",-23.541819,-46.629386
2,1/2022,2022-01-01 01:38:37,2022-01-01 00:05:00,"RUA antonio rodrigues, 1 VILA COMERCIAL PRESID...",-22.123472,-51.399824
3,2/2022,2022-01-01 02:06:38,2021-12-31 23:30:00,"AVENIDA PAULISTA, 2444 JARDIM PAULISTA S.PAULO...",-23.556360,-46.662422
4,114/2022,2022-01-01 02:33:49,2022-01-01 00:00:00,"RUA RIO DE JANEIRO, 50 LOTEAMENTO SILEX GUARUJ...",-23.996740,-46.260999
...,...,...,...,...,...,...
13495,233237/2022,2022-01-31 23:45:33,2022-01-29 00:00:00,"AVENIDA PRESTES MAIA, 0 CENTRO S.PAULO - SP",NaN,NaN
13496,233238/2022,2022-01-31 23:45:57,2022-01-31 00:00:00,"AVENIDA NORDESTINA, 5700 LAJEADO S.PAULO - SP",-23.527883,-46.417802
13497,233298/2022,2022-01-31 23:51:53,2022-01-31 00:00:00,"AVENIDA SAO JOAO, 920 REPUBLICA S.PAULO - SP",-23.540591,-46.643470
13498,233241/2022,2022-01-31 23:55:46,2022-01-31 19:30:00,"AVENIDA VINTE E TRES DE MAIO, 1000 VILA MARIAN...",-23.576289,-46.643024
